Description from `kaggle.com`

### MNIST ("Modified National Institute of Standards and Technology") 

Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike.

The goal is to correctly identify digits from a dataset of tens of thousands of handwritten images. 

Practice Skills:
- Computer vision fundamentals including simple neural networks
- Classification methods such as SVM and K-nearest neighbors

Acknowledgements: 
More details about the dataset, including algorithms that have been tried on it and their levels of success, can be found at http://yann.lecun.com/exdb/mnist/index.html. The dataset is made available under a Creative Commons Attribution-Share Alike 3.0 license.

### Data

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

### Data loading and preprocessing

In [3]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised=True)
# with_info = True - info about dataset
# as_supervised set to True loads the data in 2-tuple structure [input, target]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-train.tfrecord...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-test.tfrecord...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to C:\Users\cp24\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [6]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# We get the number of validation samples using mnist info (method from tensorflow datasets) as follows:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# This way is only available for tf datasets 
# To get the full number (not a float) we can change it to int:
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# We do the same with the number of test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# We prepare a function to scale images:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

### Modeling

In [7]:
input_size = 784 
output_size = 10 # 10 digits: 0-9
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])